In [5]:
!pip3 install google-generativeai pandas Pillow absl-py

In [6]:
import os
import getpass
import google.generativeai as genai
import PIL.Image
import pandas as pd
import sys
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [2]:
# Setup API key
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")
GOOGLE_API_KEY =  os.environ["GOOGLE_API_KEY"]
genai.configure(api_key=GOOGLE_API_KEY)

chat_model_name = 'gemini-1.5-flash'
embedding_model_name = 'models/embedding-001'

### Basic chat response

In [7]:
system_instruction="You are an AI assistant. Answer my questions concisely"
model = genai.GenerativeModel(model_name=chat_model_name, system_instruction=system_instruction)
response = model.generate_content("What is the meaning of life?")
print('\n' + response.text)


The meaning of life is a philosophical question with no one definitive answer. Different individuals and cultures have diverse perspectives. 



### Text and Image input

In [8]:
system_instruction="You are an AI assistant. Answer my questions concisely"
model = genai.GenerativeModel(model_name=chat_model_name, system_instruction=system_instruction)

image = PIL.Image.open('../data/images/image.png')
response = model.generate_content(["Write a short, engaging blog post based on this picture. It should include a description of the meal in the photo and talk about my journey meal prepping.", image])
print('\n', response.text)


 These delicious-looking meal prep containers are filled with teriyaki chicken, broccoli, carrots, peppers and rice. My meal prep journey has been amazing. Not only has it helped me eat healthier, but it's also saved me time and money. Plus, having pre-made meals ready to go has made it so much easier to stick to my fitness goals. If you're looking for a way to simplify your life and improve your diet, I highly recommend giving meal prep a try. 



### Chat with memory

In [9]:
system_instruction="You are an AI assistant. Answer my questions concisely"
model = genai.GenerativeModel(model_name=chat_model_name, system_instruction=system_instruction)
chat = model.start_chat(history=[])

response = chat.send_message("In one sentence, explain how a computer works to a young child.")
print('\n', response.text)
print(chat.history)

response = chat.send_message("Okay, how about a more detailed explanation to a high schooler?")
print('\n', response.text)
print(chat.history)


 A computer is like a brain that follows instructions to make pictures, sounds, and words appear on a screen. 

[parts {
  text: "In one sentence, explain how a computer works to a young child."
}
role: "user"
, parts {
  text: "A computer is like a brain that follows instructions to make pictures, sounds, and words appear on a screen. \n"
}
role: "model"
]

 Computers work by processing information, which is essentially a series of ones and zeros, through a complex network of circuits and components, enabling them to perform tasks like calculations, communication, and multimedia playback based on programmed instructions. 

[parts {
  text: "In one sentence, explain how a computer works to a young child."
}
role: "user"
, parts {
  text: "A computer is like a brain that follows instructions to make pictures, sounds, and words appear on a screen. \n"
}
role: "model"
, parts {
  text: "Okay, how about a more detailed explanation to a high schooler?"
}
role: "user"
, parts {
  text: "Com

### Get text document embeddings

In [11]:
def embed_fn(title, text):
    return genai.embed_content(model=embedding_model_name,
                                content=text,
                                task_type="retrieval_document",
                                title=title)["embedding"]

DOCUMENT1 = {
"title": "Operating the Climate Control System",
"content": "Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."}
DOCUMENT2 = {
    "title": "Touchscreen",
    "content": "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."}
DOCUMENT3 = {
    "title": "Shifting Gears",
    "content": "Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

df = pd.DataFrame(documents)
df.columns = ['Title', 'Text']

df['Embeddings'] = df.apply(lambda row: embed_fn(row['Title'], row['Text']), axis=1)
print(df)

                                  Title  \
0  Operating the Climate Control System   
1                           Touchscreen   
2                        Shifting Gears   

                                                Text  \
0  Your Googlecar has a climate control system th...   
1  Your Googlecar has a large touchscreen display...   
2  Your Googlecar has an automatic transmission. ...   

                                          Embeddings  
0  [-0.033361107, -0.021217084, -0.049581926, -0....  
1  [0.009660736, -0.030662702, -0.017281422, -0.0...  
2  [-0.04270796, -0.007160868, -0.03242516, -0.02...  
